In [ ]:
import os
os.chdir('/Users/jorgegustavorodriguezaboytes/ASDA205/')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm

In [ ]:
#from github load this dataset
username = "datagus"
repository = "statstutorial2025"
directory = "week5/ring_count_22plus.csv"
github_url = f"https://raw.githubusercontent.com/{username}/{repository}/main/{directory}"
df = pd.read_csv(github_url)
pd.set_option('display.max_columns', None)

In [ ]:
df

# Typical modeling

## Inspecting the data

In [ ]:
(
  sns.relplot(data=df,
              x="diam",
              y="rings",
              height=5,
              aspect=1.2)
  .set(title="Relation between rings and diameter")
  .set_axis_labels("diamater in meters","number of rings")
); #add a ; to remove the default printed format

## Creating dummy model

In [ ]:
def dummy_model(diameter):
    return diameter * 10

### Running the dummy model

In [ ]:
df["dummy_values"] = df["diam"].apply(dummy_model)

### Plotting dummy model

In [ ]:
(
  sns.relplot(data=df,
              x="diam",
              y="rings",
              height=5,
              aspect=1.2)
  .set(title="Relation between trees diamater and number of rings")
  .set_axis_labels("diamater in meters","number of rings")
)

plt.plot([0.25, 2], [dummy_model(0.25), dummy_model(2)], color="orange", linewidth=3)
plt.show();

### Calculating dummy residuals

In [ ]:
df["dummy_residuals"] = df["rings"] - df["dummy_values"]

## Running a simple regression model using statsmodel

### Running and checking summary

In [ ]:
X = df["diam"]
X = sm.add_constant(X)
y = df["rings"]

model = sm.OLS(y, X).fit()

In [ ]:
#whole summary
model.summary()

In [ ]:
#getting the parameters
model.params
intercept = model.params["const"]
slope = model.params["diam"]

### Running the model

In [ ]:
def reg_model(emission):
    return emission*slope + intercept

In [ ]:
df["reg_values"] = df["diam"].apply(reg_model)

### Plotting the regression model

In [ ]:
(
  sns.relplot(data=df,
              x="diam",
              y="rings",
              height=5,
              aspect=1.2)
  .set(title="Relation between trees' diamater and their number of rings")
  .set_axis_labels("diameter in meters","number of rings")
)

plt.plot([0.25, 2], [dummy_model(0.25), dummy_model(2)], color="purple")
plt.plot([0.25, 2], [reg_model(0.25), reg_model(2)], color="red")
plt.show();

### Calculating residuals

In [ ]:
df["reg_residuals"] = df["rings"] - df["reg_values"]

In [ ]:
#alternatively you can simply use
#af_df["reg_values"] = model.predict(X)
#af_df["reg_residuals"] = model.resid

In [ ]:
af_df.columns

## Inspecting residuals

#### histogram of residuals

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))

sns.histplot(data=df, 
             x="dummy_residuals", 
             color="purple", 
             alpha=0.2, 
             bins=10, 
             label="dummy residuals",
             ax=ax)



sns.histplot(data=df, 
             x="reg_residuals", 
             color="red", 
             alpha=0.9, 
             bins=10,
             label="reg residuals",
             ax=ax)

ax.legend()
ax.set_xlabel("residuals")
ax.set_title("Overlapped Distributions")
plt.show()

#### residual vs value

In [ ]:
ig, ax = plt.subplots(figsize=(7,4))

sns.scatterplot(data=df,
             x="rings",
             y="dummy_residuals", 
             color="purple", 
             alpha=0.9, 
             ax=ax)

sns.scatterplot(data=df,
             x="rings",
             y="reg_residuals", 
             color="blue", 
             alpha=0.9, 
             ax=ax)
plt.hlines(y=0,
           xmin=min(df['rings']),
           xmax=max(df['rings']),
           color='red')
ax.set_ylabel("residuals")
plt.show()

# Sci-kit learn Pipeline

In [ ]:
username = "datagus"
repository = "statstutorial2025"
directory = "week5/ring_count_22plus.csv"
github_url = f"https://raw.githubusercontent.com/{username}/{repository}/main/{directory}"
sdf = pd.read_csv(github_url)
pd.set_option('display.max_columns', None)

## Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

y= sdf[["rings"]]
X = sdf[["diam"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Predicting with the dummy model

In [ ]:
def dummy_model(diameter):
    return diameter * 10

In [ ]:
dummy_y_test_pred = dummy_model(X_test)

## Assesing the model

### Mean Absolute Error (MAE)

**Mean Absolute Error (MAE)** tells us how far, on average, our predictions are from the real values.  
It takes the absolute difference between each prediction and the actual value and averages them — lower MAE means better predictions.

In [ ]:
from sklearn.metrics import mean_absolute_error

dumb_mae = mean_absolute_error(y_true = y_test,
                               y_pred = dummy_y_test_pred)
float(round(dumb_mae,2))

### Root Mean Square Error (RMSE)
It takes the square root of the average squared differences between predicted and actual values — lower RMSE means better predictions.

In [ ]:
from sklearn.metrics import root_mean_squared_error

dumb_rmse = root_mean_squared_error(y_true = y_test,
                               y_pred = dummy_y_test_pred)
float(round(dumb_rmse,2))

### Mean Absolute Percentage

**Mean Absolute Percentage Error (MAPE)** tells us the average size of prediction errors **as a percentage** of the real values.  
It measures how large the errors are compared to the true values — lower MAPE means more accurate, percentage-based predictions.

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

dumb_mape = mean_absolute_percentage_error(y_true = y_test,
                                           y_pred = dummy_y_test_pred)
float(dumb_mape)

### Rsquared

**R²(Coefficient of Determination)** shows how much of the variation in the target variable is explained by the model.  
Values closer to 1 mean the model fits the data better, while values near 0 mean it explains very little.

In [ ]:
from sklearn.metrics import r2_score

dumb_r2 = r2_score(y_true = y_test,
                   y_pred = dummy_y_test_pred)

float(dumb_r2)

## Now with the linear regresision model

In [ ]:
from sklearn.linear_model import LinearRegression
# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)

### Assessing the models

In [ ]:
mae = mean_absolute_error(y_true = y_test,
                               y_pred = y_pred)


rmse = root_mean_squared_error(y_true = y_test,
                               y_pred = y_pred)

mape = mean_absolute_percentage_error(y_true = y_test,
                                           y_pred = y_pred)

r2 = r2_score(y_true = y_test,
                   y_pred = y_pred)


dummy_metrics_list = [dumb_mae, dumb_rmse, dumb_mape, dumb_r2]

reg_metrics_list = [mae, rmse, mape, r2]

evals = {"evaluation": ["mae", "rmse", "mape", "r2"],
        "metrics_dummy": dummy_metrics_list,
        "metrics_regmodel": reg_metrics_list}
evals_df = pd.DataFrame(evals)
evals_df = evals_df.round(2)

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
evals_df

### let's try another model - RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

y= sdf["rings"]
X = sdf[["diam"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rforest = RandomForestRegressor()

rforest.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = rforest.predict(X_test)

In [ ]:
#running the evaluation metrics

rf_mae = mean_absolute_error(y_true = y_test,
                               y_pred = y_pred)


rf_rmse = root_mean_squared_error(y_true = y_test,
                               y_pred = y_pred)

rf_mape = mean_absolute_percentage_error(y_true = y_test,
                                           y_pred = y_pred)

rf_r2 = r2_score(y_true = y_test,
                   y_pred = y_pred)

rf_metrics_list = [rf_mae, rf_rmse, rf_mape, rf_r2]

evals_df["metrics_randomforest"] = rf_metrics_list

In [ ]:
evals_df

# Exercise

Imagine you were hired as a data scientist for a environmental organization. Your job is to provide a model that predicts the change of average temperature given the CO2 emissions.